1. 문서의 내용을 읽는다 (w. Document Loaders)
2. 문서를 쪼갠다
    1. 토큰 수 초과로 답변을 생성하지 못할 수 있고,
    2. 문서가 길면 답변 생성이 오래걸림
3. 임베딩 후 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
# %pip install --upgrade --quiet docx2txt langchain-community
# %pip install -qU langchain-text-splitters
# %pip install langchain-chroma
# %pip install --upgrade pip setuptools wheel
# %pip install -U langchain langchainhub --quiet

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader("./tax.docx")
document = loader.load()

# print(document)
# len(document)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,  # 문서 쪼갤 때 하나의 chunk 가 가지는 토큰 수
    chunk_overlap = 200,  # 쪼갤 때, 별개로 짜르는 게 아니라, 겹치게 해서 split 
)

document_list = loader.load_and_split(text_splitter=text_splitter)
# print(document_list)

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv() # 환경변수 불러오기 

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large') 

In [5]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
# persist_directory : ./chroma 에 영구적으로 저장 
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retrieved_docs = database.similarity_search(query, k=3) # default 는 4 

In [6]:
print(len(retrieved_docs))
# print(retrieved_docs)

3


In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

# 페르소나를 주면 좋다? 
# [타이틀]
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

ai_message = llm.invoke(prompt)
print(ai_message.content)

연봉 5천만 원인 직장인의 소득세를 계산하기 위해서는 몇 가지 단계를 거쳐야 합니다. 한국의 소득세는 과세표준에 따라 단계적으로 세율이 적용됩니다. 아래에 간단한 계산 과정을 설명하겠습니다.

1. **총급여액에서 근로소득공제 차감**:
   - 연봉 5천만 원에 대한 근로소득공제를 차감하여 과세표준을 구합니다.
   - 근로소득공제는 과세표준이 증가함에 따라 차감되는 금액이 다르며, 이는 국세청의 "근로소득 간이세액표"를 참조하여 계산해야 합니다.

2. **과세표준에 따른 세율 적용**:
   - 과세표준에 따라 다음의 누진세율을 적용합니다:
     - 1,200만 원 이하: 6%
     - 1,200만 원 초과 4,600만 원 이하: 15%
     - 4,600만 원 초과 8,800만 원 이하: 24%
     - 8,800만 원 초과 1억 5천만 원 이하: 35%
     - 1억 5천만 원 초과 3억 원 이하: 38%
     - 3억 원 초과 5억 원 이하: 40%
     - 5억 원 초과: 42%

3. **세액공제 적용**:
   - 기본 공제, 인적 공제, 특별 공제 등 각종 공제를 고려하여 최종 세액을 계산합니다.

예를 들어, 근로소득공제를 고려했을 때 과세표준이 약 3,800만 원이라고 가정하면, 4,600만 원 이하 구간에 해당하여 15%의 세율을 적용받게 됩니다.

더 구체적인 금액을 계산하기 위해서는 실제 근로소득공제 금액과 추가적인 공제항목을 고려해야 하므로, 국세청의 근로소득 간이세액표를 참고하거나 세무 전문가의 상담을 받는 것이 좋습니다.


In [17]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

d:\Program\miniconda3\envs\study\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
d:\Program\miniconda3\envs\study\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
prompt  # ChatPromptTemplate
# You are an assistant for question-answering tasks
# retrieved context to answer the question

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [16]:
# Retrieval -> Question -> Answer 
from langchain.chains import RetrievalQA

# QA chain 만들기 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=database.as_retriever(), 
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})
print(ai_message["query"])
print(ai_message["result"])


연봉 5천만원인 직장인의 소득세는 얼마인가요?
죄송합니다. 제공된 정보에서는 연봉 5천만원인 직장인의 소득세를 계산하는 데 필요한 구체적인 세율이나 계산 방법을 포함하고 있지 않습니다. 소득세는 다양한 공제와 세율에 따라 달라질 수 있으므로, 정확한 계산을 위해서는 세무 전문가나 관련 세무 자료를 참조하는 것이 좋습니다.


In [22]:
# create_retrieval_chain 로 답변 생성
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

retriever = database.as_retriever(search_kwargs={"k": 1},)

combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

ai_message = retrieval_chain.invoke({"input": query})

print(ai_message)

{'input': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'context': [Document(metadata={'source': './tax.docx'}, page_content='가. 연금소득자의 나이에 따른 다음의 세율\n\n      \n\n    나. 삭제 <2014. 12. 23.>\n\n    다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n  5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n    가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n    나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n  6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n    가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n    나. 제21조제1항제18호 및 제21호에 따른 기타소득에 대해서는 100분의 15\n\n    다. 삭제 <2014. 12. 23.>\n\n    라. 그 밖의 기타소득에 대해서는 100분의 20\n\n  7. 퇴직소득에 대해서는 기본세율\n\n  8. 대통령령으로 정하는 봉사료에 대해서는 100분의 5\n\n  9. 대통령령으로 정하는 금융투자소득에 대해서는 100분의 20\n\n  ② 제1항에도 불구하고 다음 각 호의 이자소득 및 배당소득에 대해서는 다음 각 호에서 정하는 세율을 원천징수세율로 한다. <개정 2013. 1. 1., 2017. 12. 19., 2018. 12. 31., 2022. 12. 31.>\n\n  1. 「민사집행법」 제113조 및 같은 법 제142